This notebook prepare the catalogues that will be analysed by CIGALE for SED fitting and physical parameter estimation.

In [1]:
import numpy as np
import os
os.environ['LOG_LEVEL'] = 'INFO'

from astropy.table import Table

from herschelhelp.filters import correct_galactic_extinction
from herschelhelp.external import convert_table_for_cigale

In [2]:
SUFFIX = '20210120'
master_catalogue = Table.read("../../dmu32/dmu32_COSMOS/data/COSMOS_dr2_{}_cigale.fits".format(SUFFIX))

In [3]:
len(master_catalogue)

2009384

# Best sources

Define a good far-IR measurement as:
- an existing flux in the band;
- the flag from XID+ must not be set;
- the signal to noise ratio must be over 2.

In [4]:
good = {}
for band in [#'pacs_green', 'pacs_red', 
             'spire_250', 'spire_350', 'spire_500']:
    good[band] = (~np.isnan(master_catalogue['f_{}'.format(band)]))# &~master_catalogue['flag_{}'.format(band)])
    good[band][good[band]] &= (master_catalogue[good[band]]['f_{}'.format(band)] /
                              master_catalogue[good[band]]['ferr_{}'.format(band)] >= 2)

We will keep only sources with at leat 2 good far-IR measurements (we may actually use less sources are not all may have a redshift).

In [5]:
combined_good = np.sum(list(good.values()), axis=0) >= 2

In [6]:
print("Number of good sources: {}".format(np.sum(combined_good)))

Number of good sources: 40


In [7]:
# Only sources with at least two optical and at least two near infrared detections
optnir = ((master_catalogue['flag_optnir_det'] == 3) 
          | (master_catalogue['flag_optnir_det'] == 7))

# Main catalogue for CIGALE

In [8]:
best_catalogue = master_catalogue[combined_good].copy()

In [9]:
# Correction for galactic extinction
best_catalogue = correct_galactic_extinction(best_catalogue, inplace=True)

In [10]:
# Convertion to CIGALE format
best_catalogue = convert_table_for_cigale(best_catalogue, inplace=True, remove_zerofluxes=True)

INFO:herschelhelp.external:For 5 sources, the band omegacam_u should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 5 sources, the band megacam_u should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 2 sources, the band suprime_ib427 should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 4 sources, the band suprime_b should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 1 sources, the band suprime_ib464 should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 

## Band selection

We want to use only one filter for similar bands. We define an order of preference and set to NaN the flux in the lower prefered bands when a prefered band is available. Some band may have a 0 flux, we set there values to NaN.

In [11]:
u_bands = ["megacam_u",              "omegacam_u"]
g_bands = ["megacam_g", "suprime_g", "omegacam_g", "decam_g", "gpc1_g"]
r_bands = ["megacam_r", "suprime_r", "omegacam_r", "decam_r", "gpc1_r"]
i_bands = ["megacam_i", "suprime_i", "omegacam_i",            "gpc1_i"]
z_bands = ["megacam_z", "suprime_z",               "decam_z", "gpc1_z"]
y_bands = [             "suprime_y",                          "gpc1_y", "vista_y", "ukidss_y"]
j_bands = ["vista_j", "wircam_j",   "ukidss_j"  ]
h_bands = ["vista_h",  "wircam_h",  "ukidss_h"  ]
ks_bands = ["vista_ks", "wircam_ks" ]

def remove_unneeded_fluxes(list_of_bands):
    for band_idx, band in enumerate(list_of_bands[:-1]):
        mask = ~np.isnan(best_catalogue[band])
        for lower_band in list_of_bands[band_idx+1:]:
            best_catalogue[lower_band][mask] = np.nan
            best_catalogue["{}_err".format(lower_band)][mask] = np.nan

In [12]:
remove_unneeded_fluxes(u_bands)
remove_unneeded_fluxes(g_bands)
remove_unneeded_fluxes(r_bands)
remove_unneeded_fluxes(i_bands)
remove_unneeded_fluxes(z_bands)
remove_unneeded_fluxes(y_bands)
remove_unneeded_fluxes(j_bands)
remove_unneeded_fluxes(h_bands)
remove_unneeded_fluxes(ks_bands)

In [13]:
best_catalogue.write("data_tmp/COSMOS_dr2_cigale_best_extcor_{}.fits".format(SUFFIX), overwrite=True)

# All optnir objects with photoz

In [14]:
best_catalogue = master_catalogue[optnir].copy()

In [15]:
best_catalogue = best_catalogue[~np.isnan(best_catalogue["redshift"])]

In [16]:
print("Number of optnir sources with z-phot: {}".format(len(best_catalogue)))

Number of optnir sources with z-phot: 955662


In [17]:
# Correction for galactic extinction
best_catalogue = correct_galactic_extinction(best_catalogue, inplace=True)

In [18]:
# Convertion to CIGALE format
os.environ['LOG_LEVEL'] = 'INFO'
best_catalogue = convert_table_for_cigale(best_catalogue, inplace=True, remove_zerofluxes=True)

INFO:herschelhelp.external:For 203353 sources, the band omegacam_u should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 192973 sources, the band megacam_u should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 106246 sources, the band suprime_ib427 should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 154267 sources, the band suprime_b should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 78370 sources, the band suprime_ib464 should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:he

In [19]:
remove_unneeded_fluxes(u_bands)
remove_unneeded_fluxes(g_bands)
remove_unneeded_fluxes(r_bands)
remove_unneeded_fluxes(i_bands)
remove_unneeded_fluxes(z_bands)
remove_unneeded_fluxes(y_bands)
remove_unneeded_fluxes(j_bands)
remove_unneeded_fluxes(h_bands)
remove_unneeded_fluxes(ks_bands)

In [20]:
best_catalogue.write("data_tmp/COSMOS_dr2_cigale_optnir_extcor_{}.fits".format(SUFFIX), overwrite=True)

## All XID+ objects to understand selection 

In [21]:
xid = {}
for band in [#'pacs_green', 'pacs_red', 
    'spire_250', 'spire_350', 'spire_500']:
    xid[band] = (~np.isnan(master_catalogue['f_{}'.format(band)]))# & ~master_catalogue['flag_{}'.format(band)])
xid = np.sum(list(xid.values()), axis=0) >= 1

In [22]:
best_catalogue = master_catalogue[xid].copy()

In [23]:
best_catalogue = best_catalogue[~np.isnan(best_catalogue["redshift"])]

In [24]:
print("Number of optnir sources with z-phot: {}".format(len(best_catalogue)))

Number of optnir sources with z-phot: 620


In [25]:
# Correction for galactic extinction
best_catalogue = correct_galactic_extinction(best_catalogue, inplace=True)

In [26]:
for col in best_catalogue.colnames:
    if col.startswith('f_'):
        err_col = 'ferr' + col[1:]
        if col not in ['f_pacs_green', 'f_pacs_red', 'f_spire_250', 'f_spire_350', 'f_spire_500']:
            best_catalogue[col] = np.nan
            best_catalogue[col] = np.nan

In [27]:
# Convertion to CIGALE format
os.environ['LOG_LEVEL'] = 'INFO'
best_catalogue = convert_table_for_cigale(best_catalogue, inplace=True, remove_zerofluxes=True)

INFO:herschelhelp.external:For 67 sources, the band omegacam_u should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 64 sources, the band megacam_u should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 37 sources, the band suprime_ib427 should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 52 sources, the band suprime_b should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 28 sources, the band suprime_ib464 should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external

In [28]:
best_catalogue.write("data_tmp/COSMOS_dr2_cigale_allxid_extcor_{}.fits".format(SUFFIX), overwrite=True)

# Catalogue using spectroscopic redshift

In [29]:
best_catalogue = master_catalogue[optnir].copy()

In [30]:
best_catalogue.remove_column("redshift")
best_catalogue["zspec"].name = "redshift"

In [31]:
best_catalogue = best_catalogue[~np.isnan(best_catalogue["redshift"])]

In [32]:
print("Number of sources with z-spec: {}".format(len(best_catalogue)))

Number of sources with z-spec: 39544


In [33]:
# Correction for galactic extinction
best_catalogue = correct_galactic_extinction(best_catalogue, inplace=True)

In [34]:
# Convertion to CIGALE format
os.environ['LOG_LEVEL'] = 'INFO'
best_catalogue = convert_table_for_cigale(best_catalogue, inplace=True, remove_zerofluxes=True)

INFO:herschelhelp.external:For 100 sources, the band omegacam_u should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 94 sources, the band megacam_u should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 23 sources, the band suprime_ib427 should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 57 sources, the band suprime_b should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 13 sources, the band suprime_ib464 should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.externa

In [35]:
remove_unneeded_fluxes(u_bands)
remove_unneeded_fluxes(g_bands)
remove_unneeded_fluxes(r_bands)
remove_unneeded_fluxes(i_bands)
remove_unneeded_fluxes(z_bands)
remove_unneeded_fluxes(y_bands)
remove_unneeded_fluxes(j_bands)
remove_unneeded_fluxes(h_bands)
remove_unneeded_fluxes(ks_bands)

In [36]:
best_catalogue.write("data_tmp/COSMOS_dr2_cigale_optnir_extcor_zspec_{}.fits".format(SUFFIX), overwrite=True)